In [1]:
import pandas as pd
from scipy.interpolate import griddata
import numpy as np
from multiprocessing import Pool

In [2]:
#client = pd.read_csv("../data/client.csv")
#ePrices = pd.read_csv("../data/electricity_prices.csv")
#gasPrices = pd.read_csv("../data/gas_prices.csv")

weatherPred = pd.read_csv("../data/forecast_weather.csv")
#weatherHist = pd.read_csv("../data/historical_weather.csv")
#train = pd.read_csv("../data/train.csv")

# throw away unimportant weather data

In [3]:
weatherPred = weatherPred.loc[(weatherPred.hours_ahead > 23) & (weatherPred.hours_ahead < 48)]

### map lat/long to county by doing a interpolation to the county centers

In [10]:
#TODO use multiprocessing 

interpolatedVal = []
interpolationFeat = [
       'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component',
       'direct_solar_radiation', 'surface_solar_radiation_downwards',
       'snowfall', 'total_precipitation'
]

# Coordinates of points where you want to interpolate
interp_points = np.array([
    [59.31092599057606, 24.924347516442776],#"HARJUMAA",        # index = county id
    [58.89325336822078, 22.61868018616204], #"HIIUMAA",         # index = county id
    [59.177732657186525, 27.327483563720584], #"IDA-VIRUMAA",   # index = county id
    [58.9401960343481, 25.66666660100041],#   "JÄRVAMAA",       # index = county id
    [58.722190565952324, 26.536903689900708],#"JÕGEVAMAA",      # index = county id
    [59.265951945347595, 26.355665774943418],#"LÄÄNE-VIRUMAA",  # index = county id
    [58.90131388796995, 23.760348418177717],#"LÄÄNEMAA",         # index = county id
    [58.39642190189904, 24.550321648922793],#"PÄRNUMAA",         # index = county id
    [58.043991137798386, 27.162798202382948],#"PÕLVAMAA",        # index = county id
    [58.923144729594874, 24.70288373555266],#"RAPLAMAA",         # index = county id
    [58.39256707482195, 22.537559658585536],#"SAAREMAA",         # index = county id
    [58.392720495713554, 26.82996683757416],#"TARTUMAA",         # index = county id
    [58.5975, 24.9873], #"UNKNOWN", = center of the country      # index = county id
    [57.93303855829815, 26.14302669183717], #"VALGAMAA",         # index = county id
    [58.336987487787404, 25.558364059539933],#"VILJANDIMAA",     # index = county id
    [57.74795525256318, 26.91115291686715], #"VÕRUMAA"           # index = county id
  ])
# Convert to a 2D array
interp_points_arr = np.vstack(interp_points)

i = 0
print('unique days', len(weatherPred.forecast_datetime.unique())/24)
for time_id,time in enumerate(weatherPred.forecast_datetime.unique()):
  timeSlice = weatherPred.loc[weatherPred.forecast_datetime == time]
  points = timeSlice[['latitude', 'longitude']].values
  
  if (len(points)) != 112:
    print('more datapoints than we should have', time_id, time)
    break

  hours_ahead = timeSlice['hours_ahead'].unique()
  data_block_id = timeSlice['data_block_id'].unique()
  origin_datetime = timeSlice['origin_datetime'].unique()

  if len(hours_ahead) != 1 or len(data_block_id) != 1 or len(origin_datetime) != 1:
    print('more than one unique value', hours_ahead, data_block_id, origin_datetime)
  
  if i%(24*10)== 0 and i>0:
    print(i/24,' days done')
    if i%(24*100)== 0:
      print('intermediate saving..')
      intWeatherPred = pd.concat(interpolatedVal)
      intWeatherPred.to_csv("interpolPredWeather.csv")
      print('done')


  allIntFeat = {}
  allIntFeat['latitude'] = interp_points_arr[:,0]
  allIntFeat['longitude'] = interp_points_arr[:,1]
  allIntFeat['County'] = np.linspace(0,15,16).astype('int')
  allIntFeat['forecast_datetime'] = time
  allIntFeat['hours_ahead']       = hours_ahead[0]
  allIntFeat['data_block_id']     = data_block_id[0]
  allIntFeat['origin_datetime']   = origin_datetime[0]
  for feat in interpolationFeat:
    # Values at the known points
    values_feat = timeSlice[feat].values

    # Linear interpolation for temperature
    result = griddata(points, values_feat, interp_points, method='linear')

    allIntFeat[feat+'int'] = result
  
  df = pd.DataFrame(allIntFeat)
  interpolatedVal.append(df)
  i = i+1

intWeatherPred = pd.concat(interpolatedVal)
intWeatherPred.to_csv("interpolPredWeather.csv")


unique days 637.0
10.0  days done
20.0  days done
30.0  days done
40.0  days done
50.0  days done
60.0  days done
70.0  days done
80.0  days done
90.0  days done
100.0  days done
intermediate saving..
done
110.0  days done
120.0  days done
130.0  days done
140.0  days done
150.0  days done
160.0  days done
170.0  days done
180.0  days done
190.0  days done
200.0  days done
intermediate saving..
done
210.0  days done
220.0  days done
230.0  days done
240.0  days done
250.0  days done
260.0  days done
270.0  days done
280.0  days done
290.0  days done
300.0  days done
intermediate saving..
done
310.0  days done
320.0  days done
330.0  days done
340.0  days done
350.0  days done
360.0  days done
370.0  days done
380.0  days done
390.0  days done
400.0  days done
intermediate saving..
done
410.0  days done
420.0  days done
430.0  days done
440.0  days done
450.0  days done
460.0  days done
470.0  days done
480.0  days done
490.0  days done
500.0  days done
intermediate saving..
done
510.0 

In [8]:
i

12

In [11]:
intWeatherPred.head(10)

,latitude,longitude,County,forecast_datetime,hours_ahead,data_block_id,origin_datetime,temperatureint,dewpointint,cloudcover_highint,cloudcover_lowint,cloudcover_midint,cloudcover_totalint,10_metre_u_wind_componentint,10_metre_v_wind_componentint,direct_solar_radiationint,surface_solar_radiation_downwardsint,snowfallint,total_precipitationint
0,59.310926,24.924348,0,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,10.067237,5.037243,0.0,0.121643,0.028473,0.127933,1.727094,-4.232588,0.0,0.0,0.0,1.047715e-06
1,58.893253,22.618680,1,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,12.167244,5.159730,0.0,0.049673,0.139698,0.157873,1.119604,-7.190296,0.0,0.0,0.0,7.758976e-06
2,59.177733,27.327484,2,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,9.417725,6.740294,0.0,0.307994,0.337640,0.461532,1.945832,-3.715220,0.0,0.0,0.0,6.418636e-05
3,58.940196,25.666667,3,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,8.882337,5.618534,0.0,0.307195,0.105001,0.352315,2.174265,-3.226562,0.0,0.0,0.0,5.673582e-06
4,58.722191,26.536904,4,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,8.830031,5.928326,0.0,0.123666,0.123649,0.203788,2.592233,-3.410328,0.0,0.0,0.0,5.755532e-06
5,59.265952,26.355666,5,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,8.917012,5.843676,0.0,0.261071,0.196946,0.357129,1.771527,-3.480260,0.0,0.0,0.0,6.188323e-05
6,58.901314,23.760348,6,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,10.284310,4.881773,0.0,0.011317,0.003629,0.012775,1.178737,-4.379696,0.0,0.0,0.0,1.579015e-07
7,58.396422,24.550322,7,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,9.832134,5.690962,0.0,0.047184,0.028792,0.055189,2.308487,-4.575043,0.0,0.0,0.0,0.000000e+00
8,58.043991,27.162798,8,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,8.743266,5.470778,0.0,0.224027,0.154159,0.260906,2.814128,-3.084321,0.0,0.0,0.0,2.125580e-05
9,58.923145,24.702884,9,2021-09-02 00:00:00+00:00,24,1,2021-09-01 00:00:00+00:00,9.321824,4.818816,0.0,0.078133,0.022398,0.082504,2.001133,-3.420197,0.0,0.0,0.0,0.000000e+00
